In [56]:
# imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# Last inn data
purchase_orders = pd.read_csv("data/kernel/purchase_orders.csv", parse_dates=["delivery_date", "created_date_time", "modified_date_time"])
receivals = pd.read_csv("data/kernel/receivals.csv", parse_dates=["date_arrival"])
raw_material = pd.read_csv("data/extended/materials.csv")
transportation = pd.read_csv("data/extended/transportation.csv")


purchase_orders_clean = purchase_orders[purchase_orders['quantity'] > 0]
purchase_orders_clean = purchase_orders_clean[purchase_orders_clean['status'] != 'Deleted']
receivals_clean = receivals[receivals['net_weight'] > 0]

purchase_base = purchase_orders_clean[["purchase_order_id", 
                                       "quantity",
                                       "created_date_time",
                                       "purchase_order_item_no",
                                       ]]

receivals_base = receivals_clean[["rm_id",
                                 "purchase_order_id",
                                 "purchase_order_item_no",
                                 "product_id",
                                 "date_arrival",
                                 "net_weight"
                                 ]]

rec_ord_merged = receivals_base.merge(
    purchase_base,
    on = ["purchase_order_id", "purchase_order_item_no"],
    how = "inner"
    )



   rm_id  purchase_order_id  purchase_order_item_no  product_id  \
0  365.0           208545.0                    10.0  91900143.0   
1  365.0           208545.0                    10.0  91900143.0   
2  365.0           208490.0                    10.0  91900143.0   
3  365.0           208490.0                    10.0  91900143.0   
4  379.0           210435.0                    20.0  91900296.0   
5  389.0           208535.0                    30.0  91900330.0   
6  365.0           208532.0                    10.0  91900143.0   
7  369.0           208532.0                    30.0  91900146.0   
8  366.0           208532.0                    20.0  91900160.0   
9  365.0           208537.0                    10.0  91900143.0   

                date_arrival  net_weight   quantity         created_date_time  
0  2004-06-15 13:34:00+02:00     11420.0  1975000.0 2004-01-13 15:30:57+00:00  
1  2004-06-15 13:34:00+02:00     13760.0  1975000.0 2004-01-13 15:30:57+00:00  
2  2004-06-15 13:38:00

In [78]:
df = rec_ord_merged.copy()

for c in ["date_arrival", "created_date_time"]:
    df[c] = pd.to_datetime(df[c], errors="coerce", utc=True).dt.tz_localize(None)
    
df["delivery_year"]    = df["date_arrival"].dt.year
df["delivery_month"]   = df["date_arrival"].dt.month
df["delivery_weekday"] = df["date_arrival"].dt.weekday
df["lead_time_days"] = (df["date_arrival"] - df["created_date_time"]).dt.days



In [79]:
weight_diff = df["net_weight"] - df["quantity"]
df["delivery_ratio"] = df["net_weight"] / df["quantity"]

features = [
    'rm_id',
    'product_id',
    'created_date_time',
    'net_weight',
    'quantity',
    'date_arrival',
    'delivery_year',
    'delivery_month',
    'delivery_weekday',
    'lead_time_days',
    'weight_diff',
    'delivery_ratio'
]

In [77]:
""" # Features vi vil ha fra purchase_orders
purchase_orders_base = purchase_orders_clean[
    [
        "purchase_order_id",
        "purchase_order_item_no",
        "quantity",
        "product_id",
        "product_version",
        "status",
        "created_date_time"
    ]
].copy()

# Leveranser per ordrelinje
receivals_grouped_by_lines = (
    receivals_clean
    .groupby(["purchase_order_id", "purchase_order_item_no"], as_index=False)
    .agg(
        delivered=("net_weight", "sum"),
        n_receivals=("receival_item_no", "nunique"),
        delivery_start=("date_arrival", "min"),
        rm_id_line=("rm_id", "first")  
    )
)

# Slår sammen purchase_orders med leveranser per linje
pur_rec_merged_by_line = purchase_orders_base.merge(
    receivals_grouped_by_lines,
    on=["purchase_order_id", "purchase_order_item_no"],
    how="inner",
    validate="one_to_one"
)

# Fyll manglende leveranser med 0 og sikre typer
pur_rec_merged_by_line["delivered"] = pur_rec_merged_by_line["delivered"].fillna(0.0)
pur_rec_merged_by_line["n_receivals"] = pur_rec_merged_by_line["n_receivals"].fillna(0).astype(int)

# 5. Linje-fill rate (leverte kg / bestilt kg)
pur_rec_merged_by_line["line_fill_ratio"] = pur_rec_merged_by_line["delivered"] / pur_rec_merged_by_line["quantity"]

# --- Ny aggregering: per (purchase_order_id, rm_id_line) ---
order_rm = (
    pur_rec_merged_by_line
    .dropna(subset=["rm_id_line"])  # hvis noen linjer mangler rm_id
    .groupby(["purchase_order_id", "rm_id_line"], as_index=False)
    .agg(
        ordered_quantity=("quantity", "first"),
        delivered_total=("delivered", "sum"),
        n_lines=("purchase_order_item_no", "nunique"),
        n_lines_with_delivery=("delivered", lambda s: (s > 0).sum()),
        time_created_order=("created_date_time", "min"),
        time_first_delivery=("delivery_start", "min"),
        status_po=("status", "first"),
        # nyttig: saml unike product_ids for akkurat denne (PO, rm)-kombinasjonen
        product_ids=("product_id", lambda s: sorted(set(s)))
    )
)

# Ratioer på (PO, rm)-nivå
order_rm["delivery_ratio"] = order_rm["delivered_total"] / order_rm["ordered_quantity"]
order_rm["line_delivery_fraction"] = order_rm["n_lines_with_delivery"] / order_rm["n_lines"]

# Ekstremfilter per (PO, rm)
threshold = 10
order_rm["extreme_flag"] = order_rm["delivery_ratio"] > threshold
order_rm_clean = order_rm.loc[~order_rm["extreme_flag"]].copy().reset_index(drop=True)

# Ryddige kolonnenavn
order_rm_clean = order_rm_clean.rename(columns={"rm_id_line": "rm_id"})

print(order_rm_clean.head(10)) """

' # Features vi vil ha fra purchase_orders\npurchase_orders_base = purchase_orders_clean[\n    [\n        "purchase_order_id",\n        "purchase_order_item_no",\n        "quantity",\n        "product_id",\n        "product_version",\n        "status",\n        "created_date_time"\n    ]\n].copy()\n\n# Leveranser per ordrelinje\nreceivals_grouped_by_lines = (\n    receivals_clean\n    .groupby(["purchase_order_id", "purchase_order_item_no"], as_index=False)\n    .agg(\n        delivered=("net_weight", "sum"),\n        n_receivals=("receival_item_no", "nunique"),\n        delivery_start=("date_arrival", "min"),\n        rm_id_line=("rm_id", "first")  \n    )\n)\n\n# Slår sammen purchase_orders med leveranser per linje\npur_rec_merged_by_line = purchase_orders_base.merge(\n    receivals_grouped_by_lines,\n    on=["purchase_order_id", "purchase_order_item_no"],\n    how="inner",\n    validate="one_to_one"\n)\n\n# Fyll manglende leveranser med 0 og sikre typer\npur_rec_merged_by_line["deli

In [ ]:

""" cat_cols = ["product_ids"]
encoders = {}
for col in cat_cols:
    df[col] = df[col].astype(str).fillna("MISSING")
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le """
    


In [ ]:
features = [
    "ordered_quantity",
    "delivered_total",  
    "rm_ids",
    "delivery_year",
    "delivery_month", 
    "delivery_weekday",
    "lead_time_days",
    "product_ids",
    "delivery_ratio",
    "created_date_time"
]

x = df[features].copy()
y = df["delivered_total"].values


Features shape: (7150, 13)
   ordered_quantity  delivered_total  delivery_ratio  line_delivery_fraction  \
0          300000.0          20400.0        0.068000                     1.0   
1          150000.0           2460.0        0.016400                     1.0   
2          150000.0           6340.0        0.042267                     1.0   
3          640660.0         107270.0        0.167437                     1.0   
4         1701000.0         918561.0        0.540012                     1.0   

   n_rm_ids  n_product_ids  rm_ids_sig  product_ids_sig  lead_time_days  \
0         2              0    0.427668              0.0               1   
1         1              0    0.214438              0.0               8   
2         1              0    0.214438              0.0               3   
3         2              0    0.809885              0.0             460   
4         2              0    0.740628              0.0             158   

   delivery_year  delivery_month  deliver

C:\Users\strom\AppData\Local\Temp\ipykernel_30508\2344725399.py:23: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['created_epoch_days'] = (df['created_date_time'].view('int64') // 10**9) / 86400.0


In [41]:
train_mask = df["time_first_delivery"] < pd.Timestamp("2024-01-01")

X_train, y_train = x[train_mask], y[train_mask]
X_test,  y_test  = x[~train_mask], y[~train_mask]

# Behold ID-informasjonen også
test = df.loc[~train_mask].copy()

print(X_test.head(100))

      ordered_quantity  delivered_total                    rm_ids  \
6580           25000.0          24452.0                  [2130.0]   
6661            3240.0           3240.0                  [2124.0]   
6662          150000.0         140580.0                  [2741.0]   
6663          400000.0         388940.0                  [3125.0]   
6664          225000.0         230400.0                  [3126.0]   
...                ...              ...                       ...   
6766           65000.0         164637.0  [2142.0, 2144.0, 2145.0]   
6767          300000.0         705674.0                  [3781.0]   
6768           25000.0          24236.0                  [3362.0]   
6769           25000.0          95040.0                  [3865.0]   
6770          300000.0         122967.0                  [3781.0]   

      delivery_year  delivery_month  delivery_weekday  lead_time_days  \
6580           2024               1                 4             116   
6661           2024      

In [39]:
model = GradientBoostingRegressor(
    loss = 'quantile',
    alpha = 0.2,
    n_estimators=300,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    min_samples_leaf=10,
    subsample=0.8,
)  

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")


ValueError: setting an array element with a sequence.

In [ ]:
""" from kaggle_metric import score, ParticipantVisibleError


receivals["date_arrival"] = pd.to_datetime(receivals["date_arrival"], errors="coerce", utc=True).dt.tz_localize(None)


start = pd.Timestamp("2024-01-01")
end   = pd.Timestamp("2024-05-31 23:59:59")
solution = (receivals.loc[(receivals["date_arrival"] >= start) & (receivals["date_arrival"] <= end)]
            .groupby("rm_id", as_index=False)
            .agg(weight=("net_weight", "sum"))
           ).rename(columns={"rm_id": "ID"})


rm_col = "rm_id_raw" if "rm_id_raw" in test.columns else "rm_id"

preds = pd.DataFrame({
    "ID": test[rm_col].values,
    "predicted_weight": np.clip(y_pred, 0, None)  
})

submission = preds.groupby("ID", as_index=False).agg(predicted_weight=("predicted_weight", "sum"))


submission = solution[["ID"]].merge(submission, on="ID", how="left")
submission["predicted_weight"] = submission["predicted_weight"].fillna(0.0)


try:
    final_score = score(solution=solution, submission=submission, row_id_column_name="ID")
    print("Quantile loss (q=0.2) – backtest jan–mai 2024:", final_score)
except ParticipantVisibleError as e:
    print("Scoring feilet:", e)
    

print(submission.head())
print(solution.head()) """

' from kaggle_metric import score, ParticipantVisibleError\n\n\nreceivals["date_arrival"] = pd.to_datetime(receivals["date_arrival"], errors="coerce", utc=True).dt.tz_localize(None)\n\n\nstart = pd.Timestamp("2024-01-01")\nend   = pd.Timestamp("2024-05-31 23:59:59")\nsolution = (receivals.loc[(receivals["date_arrival"] >= start) & (receivals["date_arrival"] <= end)]\n            .groupby("rm_id", as_index=False)\n            .agg(weight=("net_weight", "sum"))\n           ).rename(columns={"rm_id": "ID"})\n\n\nrm_col = "rm_id_raw" if "rm_id_raw" in test.columns else "rm_id"\n\npreds = pd.DataFrame({\n    "ID": test[rm_col].values,\n    "predicted_weight": np.clip(y_pred, 0, None)  \n})\n\nsubmission = preds.groupby("ID", as_index=False).agg(predicted_weight=("predicted_weight", "sum"))\n\n\nsubmission = solution[["ID"]].merge(submission, on="ID", how="left")\nsubmission["predicted_weight"] = submission["predicted_weight"].fillna(0.0)\n\n\ntry:\n    final_score = score(solution=solution,